In [16]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [2]:
# O esquemático básico é:

# pop = init_pop_rand
# avalia_pop(pop)
# while condicao:
#     t += 1
#     selec_pai(pop)
#     crossover
#     mutacao
#     avalia_pop(pop)

In [ ]:
def funcao_a_minimizar(x):
    return x**2 + 0.3*np.cos(10*np.pi*x)

def init_indv_rand(num_bits):
    return [int(bool(b)) for b in np.random.randint(2, size=num_bits)]

def bin_from_gray(g):
    b = [g[-1]]
    for n in g[-2::-1]:
        b.insert(0, b[0]^n)
    return b
    
def fenotipo(cromossomo, num_bits):
    b = bin_from_gray(cromossomo)
    u = sum(2**k for k in range(num_bits) if b[k])
    u /= 2**num_bits
    return -2 + 4 * u
    
def constroi_pop_rand(num_indv,num_bits):
    return np.array(list(map(lambda i: init_indv_rand(num_bits),range(num_indv))))

def calc_fitness(populacao,num_indv,num_bits):
    return list(map(lambda i: funcao_a_minimizar(fenotipo(list(populacao[i,:]),num_bits)),range(num_indv)))

def parent_selection(populacao,fitness,num_parents):
    a = np.sort(fitness/max(fitness))
    mating_pool = []
    current_member = 1
    ue = 1
    r = np.random.uniform(0,1/num_parents)
    while (current_member <= num_parents):
        while (r <= a[ue]):
            mating_pool.append(populacao[ue].tolist())
            r = r + 1/num_parents
            current_member = current_member + 1
        ue = ue + 1
    return np.array(mating_pool)

def swap_mutation(genotipo):
    i, j = np.random.choice(range(len(genotipo)),2)
    perm = genotipo
    perm[i], perm[j] = perm[j], perm[i]
    return perm

def apply_swap_mutation(mating_pool,num_parents):
    return np.array(list(map(lambda i: swap_mutation(mating_pool[i,:]),range(num_parents))))

# def single_pmx(chr0, chr1, i, j):
#         child = [None for _ in chr0]
#         child[i:j] = chr0[i:j]
#         for k in range(i,j):
#             g = chr1[k]
#             if g in child:
#                 continue
#             l = chr1.index(chr0[k])
#             while not (child[l] is None):
#                 l = chr1.index(chr0[l])
#             child[l] = g
#         for k in range(len(child)):
#             if child[k] is None:
#                 child[k] = chr1[k]
#         return child

# def recomb_pmx(chromossomes):
#     """Partially mapped crossover"""
#     i, j = random.sample(range(len(chromossomes[0])), 2)
#     if i > j: i,j = j,i
#     j += 1
#     children = [None for _ in chromossomes]
#     ch = random.getrandbits(1)
#     children[ch^0] = single_pmx(list(chromossomes[0]), list(chromossomes[1]), i, j)
#     children[ch^1] = single_pmx(list(chromossomes[1]), list(chromossomes[0]), i, j)
#     return np.array(children)
    
    
num_bits = 16
num_indv = 10
num_parents = 4

# inicializa a populacao de forma aleatoria
populacao = constroi_pop_rand(num_indv,num_bits)

# avalia a populacao inicial
fitness = calc_fitness(populacao,num_indv,num_bits)
mating_pool = parent_selection(populacao,fitness,num_parents)
fitness_pais= calc_fitness(mating_pool,len(mating_pool),num_bits)
dps_mutacao = apply_swap_mutation(mating_pool,num_parents)
children = recomb_pmx(dps_mutacao)
children